In [1]:
from unis.models import *
from unis.runtime import Runtime
from unis.rest import UnisReferenceError
from unis.models.lists import UnisCollection

In [2]:
local = "http://192.168.100.100:8888"
local_source = "http://192.168.100.200:8888"
sushivan = "http://149.160.232.188:8888"
dev = "http://dev.crest.iu.edu:8888"

In [3]:
def runtime_copy(source, target):
    for collection in source.collections:
        for item in getattr(source, collection):
            model = type(item)
            copy = model(item.to_JSON())
            target.insert(copy)

def commit(target):
    collections = ["ports", "nodes", "links", "paths", "networks", "domains", "topologies", "exnodes", "extents"]
    unordered = filter(lambda x: x not in collections, target.collections)
    collections.extend(unordered)
    for collection in collections:
        for item in getattr(target, collection):
            print("Commiting", type(item), item.id)
            item.commit()

def copy(source_url, target_url):
    with Runtime(source_url) as source, Runtime(target_url) as target:
        runtime_copy(source, target)
        patch_refs(source, target)
        commit(target)

In [4]:
def patch_collection_refs(source, target, collection):
    try:
        i = 0
        while True:
            try:
                entry=collection[i]
                i = i+1
            except UnisReferenceError as exp:
                print(exp.href)
                src = source.find(exp.href)
                print("Patching ", src.id)
                dst=list(dst.ports.where({"id": p.id}))[0]
                collection[i]=dst
    except IndexError:
        return
    
def patch_refs(source, target):
    for collection in target.collections:
        for item in getattr(target, collection):
            for att in dir(item):
                try:
                    entry = item.__getattribute__(att)
                    if isinstance(entry, UnisCollection):
                        patch_collection_refs(source, target, entry)
                except Exception as exp:
                    print("Error: ", exp, item.id, type(item), att)

In [121]:
copy(sushivan, local)

Commiting <class 'unis.models.models.Port'> 57d5a305b23436192f78c795
Commiting <class 'unis.models.models.Port'> 57d5a305b23436192f78c794
Commiting <class 'unis.models.models.Port'> 57d5a305b23436192f78c793
Commiting <class 'unis.models.models.Port'> 57d5a305b23436192f78c77d
Commiting <class 'unis.models.models.Port'> 57d5a305b23436192f78c773
Commiting <class 'unis.models.models.Port'> 57d5a305b23436192f78c76c
Commiting <class 'unis.models.models.Port'> 57d5a305b23436192f78c762
Commiting <class 'unis.models.models.Port'> 57d5a305b23436192f78c758
Commiting <class 'unis.models.models.Port'> 57d5a305b23436192f78c74e
Commiting <class 'unis.models.models.Port'> 57d5a304b23436192f78c744
Commiting <class 'unis.models.models.Node'> 57d5a305b23436192f78c78c
Commiting <class 'unis.models.models.Node'> 57d5a305b23436192f78c78b
Commiting <class 'unis.models.models.Node'> 57d5a305b23436192f78c78a
Commiting <class 'unis.models.models.Node'> 57d5a305b23436192f78c77a
Commiting <class 'unis.models.mode